In [2]:
import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn
import json
import pandas as pd

In [3]:
# ONS Roundup Wayback machine archive urls
url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26&collapse=digest&from=20200101&to=20210521&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls) #parses the JSON from urls.

In [5]:
## Extracts timestamp and original columns from urls and compiles a url list.
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
## Compiles final url pattern.
final_urls = []
for url in url_list:
    final_urls.append('https://web.archive.org/web/'+url)

In [1426]:
# ONS LATEST INSIGHTS APRIL Wayback machine archive urls
# url = 'http://web.archive.org/cdx/search/cdx?url=https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights&collapse=digest&from=20210709&to=20210801&output=json'
url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26&collapse=digest&from=20200801&to=20210101&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls) #parses the JSON from urls.

## Extracts timestamp and original columns from urls and compiles a url list.
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
## Compiles final url pattern.
final_urls = []
for url in url_list:
    final_urls.append('https://web.archive.org/web/'+url)

In [1427]:
len(final_urls)

71

In [1428]:
def getpieces(url):
    # Open page for roundup
    req = rq.get(url).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')

    # Find the section elements from the page
    sections = soup.find_all('section')
    # Filter out ones without h3 elements (headlines change between h3 and h2)
    pieces = []
    for section in sections:
        if section.find_all('h3') != []:
            pieces.append(section)

    pieces = [i for i in pieces if i.find('h3').text not in ["Latest COVID-19 headlines", "Explore by topic", "Also published recently"]]
    return pieces


In [1429]:
# This is a function to format a single roundup entry 
def makeru(p):
        
    # Put in a list and remove line breaks
    p = [i for i in p if i not in ['\n']]
    p = [i for i in p if (i.text not in ['Analysis | Data', 'Download the data'])&('Embed this interactive' not in i.text)]
    
    p2 = []
    last = 'Not Notes'
    for n, i in enumerate(p):
        if (len(i.text) < 20) & (n < 3):
            pass
        else:
            if i.text == 'Notes':
                last = 'Notes'
            else:
                if last != 'Notes':
                    p2.append(i)
                last = 'Not Notes'
                
    
    # Remove all h4s and h5s which are the dates and chart titles, filter out elements that contain text suggesting its furniture
    p2 = [i for i in p2 if (i.name not in ['h2', 'h4', 'h5', 'summary', 'div']) and ('Embed code' not in i.text) and ('XLSX' not in i.text) and ('Explore the latest' not in i.text) and ('Read the latest' not in i.text) and ('Read the full bulletin' not in i.text) and ('Back to the top' not in i.text) and ('Download this chart' not in i.text)]
        
    # For each element in the section, if it is a list iteratre the list adding a dash to the listed text, if not a list then just get text from element
    p2 = [ [ " - " + i.text for i in q if i != '\n' ] if q.name == 'ul' else q.text for q in p2 ]

    string = ""
    
    for i in p2:
        if type(i) == list:
            for j in i:
                string = string + j + '\n'
        else:
            string = string + i + '\n'
        
    return string

print(makeru(pieces[5]))

The employment rate was falling sharply and unemployment was rising in the three months to October
Early estimates for November 2020 suggest that there is a slight drop over the month in the number of payroll employees in the UK. Since February 2020, the number of payroll employees has fallen by 819,000; however, the larger falls were seen at the start of the coronavirus (COVID-19) pandemic.
Data from our Labour Force Survey show a large increase in the unemployment rate while the employment rate continues to fall.
In the three months to October 2020, redundancies reached a record high of 370,000; an increase of a record 217,000 on the quarter, although the number of redundancies fell slightly in October.
The number of people temporarily away from work has fallen since its peak in April and May 2020. The number of people away from work because of the pandemic and receiving no pay has also fallen and levelled off in recent months.
Annual growth in employee pay continued to strengthen as

In [1430]:
def getbulletinurl(piece):

    p = [i for i in piece if i != '\n']
                
    try:
        p = [i for i in p if ('Read the bulletin' in i.text)|('Explore the latest insights' in i.text)|('Read the full' in i.text)|('Read the latest' in i.text)][0]
        if len(p.find_all('li'))>0:
            p = p.find_all('li')
            p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
        p = p.find_all('a')

    except:
        try:
            p = [i for i in p if ('Read more' in i.text)|('Full bulletin' in i.text)][0]
            if len(p.find_all('li'))>0:
                p = p.find_all('li')
                p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
            p = p.find_all('a')
        except:
            try:
                p = [i for i in p if ('Get the' in i.text)|('Article on' in i.text)|('Read' in i.text)|('Latest bulletin' in i.text)][0]
                if len(p.find_all('li'))>0:
                    p = p.find_all('li')
                    p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
                p = p.find_all('a')
            except:
                try:
                    p = [i for i in p if 'Latest real-time indicators' in i.text][0]
                    if len(p.find_all('li'))>0:
                        p = p.find_all('li')
                        p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
                    p = p.find_all('a')
                except:
                    try:
                        p = [i for i in p if ('Explore the latest indicators' in i.text)|('Analysis' == i.text)][0]
                        if len(p.find_all('li'))>0:
                            p = p.find_all('li')
                            p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
                        p = p.find_all('a')
                    except:
                        return "NO LINK"

    p = p[0]['href']
    p = "http" + p.split("http")[-1]
    p = p.replace("https:/www", "https://www")
        
    date = piece.find('p').text
    reldate = "Released: " + date
#     date = piece.find('h4').text
    
                    
    if p.split("/")[-1] == 'latest':
                
        req = rq.get(p).text
        soup = bs(req,'html.parser')

        prevrelurl = "https://www.ons.gov.uk" + [i for i in soup.find_all("a") if i.text =='View previous releases'][0]['href']
                
#         if piece.find('h3').text == 'UK online job adverts in catering and hospitality rise above pre-pandemic levels':
#             prevrelurl = "https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/previousReleases"
         
#         if piece.find('h3').text == 'UK seated diner reservations increase by 13 percentage points':
#             prevrelurl = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'
        
        req = rq.get(prevrelurl).text
        soup = bs(req,'html.parser')

        lis = soup.find("main").find("div", class_="print-links").find_all('li')
                
        try:
            linkseg = [i.find("a") for i in lis if date in i.text][0]['href']
        except:
            
            for i in range(1, 20):
                req = rq.get( prevrelurl + "?page=" + str(i) ).text
                soup = bs(req,'html.parser')
                
                if '404 - ' not in soup.find('title').text:
                    lis = soup.find("main").find("div", class_="print-links").find_all('li')

                    if len([i.find("a") for i in lis if reldate.rstrip() in i.text]) > 0:
                        linkseg = [i.find("a") for i in lis if reldate.rstrip() in i.text][0]['href']
                        break
                        
            if 'Coronavirus and the social impacts' in soup.find('h1').text:
                prevrelurl = "https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/previousReleases"

            elif 'Economic activity and social change' in soup.find('h1').text:
                prevrelurl = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'

            for i in range(1, 20):
                req = rq.get( prevrelurl + "?page=" + str(i) ).text
                soup = bs(req,'html.parser')
                
                if '404 - ' not in soup.find('title').text:
                    lis = soup.find("main").find("div", class_="print-links").find_all('li')

                    if len([i.find("a") for i in lis if reldate.rstrip() in i.text]) > 0:
                        linkseg = [i.find("a") for i in lis if reldate.rstrip() in i.text][0]['href']
                        break

        bulletinurl = "https://www.ons.gov.uk" + linkseg
        
    elif '/releases/' not in p:
        bulletinurl = p
    else:    
        req = rq.get(p).text
        # parse roundup html using beautifulsoup and store in soup
        soup = bs(req,'html.parser')
        
        cols = soup.find_all('div', class_="col-wrap")
        
        pubs = [i for i in cols if 'Publications' in i.text]
        
        if len(pubs)>0:
            pub = pubs[0]
            pubind = cols.index(pub)
            linksect = cols[pubind+1]
            bulletinurl = "https://www.ons.gov.uk" + linksect.find("a")['href']
            
        else:
            bulletinurl = "NO PUBLICATION"
            
    bulletinurl = bulletinurl.replace("https://ons", "https://www.ons")
    
    return bulletinurl

getbulletinurl(pieces[4])

'NO LINK'

In [1472]:
def getbull(bulletinurl):
    
    # Open page for roundup
    req = rq.get(bulletinurl).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')
    
    article = soup.find('article')
        
    bull = []
    
    for el in article:
        if el != '\n':
            if el.find('h2') != None:
                print(el) 
                if 'Other pages' not in el.find('h2').text:
                    print(" * * * ")
                    bull.append(el)
    
    return bull

print(getbull('https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/uklabourmarket/june2021'))

<div class="section__content--markdown" id="other-pages-in-this-release">
<section>
<header>
<h2><span class="section__content-number">1.</span> Other pages in this release</h2>
</header>
<ul>
<li><p><a href="https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/averageweeklyearningsingreatbritain/latest">Average weekly earnings in Great Britain</a></p></li>
<li><p><a href="https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/latest">Earnings and employment from Pay As You Earn Real Time Information, UK</a></p></li>
<li><p><a href="https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/employmentintheuk/latest">Employment in the UK</a></p></li>
<li><p><a href="https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/regionallabourmarket/latest">Labour marke

In [1432]:
def addtodf(pieces, df):
    for i, piece in enumerate(pieces):
        if makeru(piece) not in [i for i in df['roundup']]:
            if 'ons.gov.uk' in getbulletinurl(piece):
                print( ' url ', getbulletinurl(piece))
                if getbulletinurl(piece) != 'NO PUBLICATION':
                    if getbulletinurl(piece) == 'NO LINK':
                        print(" :: NO LINK :: ", piece)
                        nolinks.append(piece)
                    else:
                        df = df.append({'roundup': makeru(piece), 'bulletin_url': getbulletinurl(piece), 'bulletin_mps': getmps(getbulletinurl(piece))}, ignore_index=True)
                        print(' ** ADDED ** ', makeru(piece))
                else:
                    print(getbulletinurl(piece), piece)
            else:
                print( ' :: WEIRD URL :: ', piece)
                weirdurls.append(piece)

    return df


In [1433]:
# d = {'roundup': [], 'bulletin_url': [], 'bulletin_mps': []}
# df = pd.DataFrame(data=d)

for i in range(1, len(final_urls)+1):
    print('pieces', i)
    print(final_urls[-i])
    pieces = getpieces(final_urls[-i])
    df = addtodf(pieces, df)
df

pieces 1
https://web.archive.org/web/20210101230434/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26
 :: WEIRD URL ::  <section>
<h2 id="health">Infections and deaths</h2><h3>COVID-19 positivity rates increasing in England, Wales and Northern Ireland</h3><p><i>24 December 2020</i></p><p>The <a href="https://web.archive.org/web/20210101230434/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronaviruscovid19infectionsurveypilot/latest">number of people with the coronavirus (COVID-19)</a> in England has increased, with 645,800 people estimated to have had COVID-19 in the most recent week (12 to 18 December 2020). This equates to 1 in 85 people. </p><p>The positivity rate in Wales has risen sharply, with 1 in 60 people estimated to have had COVID-19 in the most recent week. There has also been an increase in Northern Ireland, but the rate there

UnboundLocalError: local variable 'linkseg' referenced before assignment

In [1473]:
df

,roundup,bulletin_url,bulletin_mps
0,Infections remain high in Wales but fall in ot...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, the percentage of people testing p..."
1,No evidence suicides increased during the COVI...,https://www.ons.gov.uk/peoplepopulationandcomm...,"3,828 suicides occurred between April and Dece..."
2,Retail and recreation visits remain below pre-...,https://www.ons.gov.uk/economy/economicoutputa...,Most of the consumer behaviour measures we mon...
3,Vaccinated remain less likely to test positive...,https://www.ons.gov.uk/peoplepopulationandcomm...,Most people who reported having been vaccinate...
4,Deaths in care homes below five-year average\n...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In the week ending 1 April 2022 (Week 13), 9,8..."
...,...,...,...
383,Coronavirus hits young and poorly-paid furloug...,https://www.ons.gov.uk/peoplepopulationandcomm...,The labour market shocks associated with the c...
384,UK debit and credit card purchases 35% below t...,https://www.ons.gov.uk/peoplepopulationandcomm...,Today we are launching an experimental faster ...
385,COVID-19 infections and hospital admissions in...,https://www.ons.gov.uk/peoplepopulationandcomm...,It is estimated that 1 in 50 people in England...
386,Retail footfall reaches lowest levels since mi...,https://www.ons.gov.uk/peoplepopulationandcomm...,Local and national restrictions changed during...


In [1434]:
len(weirdurls)

485

In [1415]:
len(nolinks)

0

In [1474]:
import random

In [1503]:
roundupsum_train = {}
roundupsum_train['data'] = []

roundupsum_val = {}
roundupsum_val['data'] = []

In [1504]:
for i in df.index:
    if random.random() < 0.1:
        roundupsum_val['data'].append({
            'document': df.iloc[i]['bulletin_mps'],
            'summary': df.iloc[i]['roundup'],
            'id': i
        })
    else:
        roundupsum_train['data'].append({
            'document': df.iloc[i]['bulletin_mps'],
            'summary': df.iloc[i]['roundup'],
            'id': i
        })

In [1505]:
with open('roundupsum_train.json', 'w') as outfile:
    json.dump(roundupsum_train, outfile)
    
with open('roundupsum_val.json', 'w') as outfile:
    json.dump(roundupsum_val, outfile)

In [1451]:
!pwd

/Users/theojolliffe/Documents/Roundup Scrape


In [1437]:
df.to_csv('roundup.csv')

In [1244]:
nolinks = []

In [1247]:
weirdurls = []

In [ ]:
# ORIGINAL!!!!!
def makeru(p):
    print(p)
    
    # Put in a list and remove line breaks
    p = [i for i in p if i not in ['\n']]
    p = [i for i in p if (i.text not in ['Analysis | Data', 'Notes', 'Download the data'])&('Embed this interactive' not in i.text)]

    print(p[-4])
    
    # Remove all h4s and h5s which are the dates and chart titles, filter out elements that contain text suggesting its furniture
    p = [i for i in p if (i.name not in ['h4', 'h5', 'summary', 'div']) and ('Embed code' not in i.text) and ('XLSX' not in i.text) and ('Explore the latest' not in i.text) and ('Read the latest' not in i.text) and ('Read the full bulletin' not in i.text) and ('Back to the top' not in i.text) and ('Download this chart' not in i.text)]
        
    # For each element in the section, if it is a list iteratre the list adding a dash to the listed text, if not a list then just get text from element
    p = [ [ " - " + i.text for i in q if i != '\n' ] if q.name == 'ul' else q.text for q in p ]

        
    string = ""
    
    for i in p:
        if type(i) == list:
            for j in i:
                string = string + j + '\n'
        else:
            string = string + i + '\n'
        
    return string

makeru(pieces[2])

In [ ]:
def getmps(bulletinurl):
    
    # Open page for roundup
    req = rq.get(bulletinurl).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')
        
#     soup = [i for i in soup.find('section') if ((i.name == 'ul')|(i.name == 'p'))]
#     print('soup', soup)
#     mps = [i.text for i in soup if ('Source: Office for National Statistics' not in i.text) and ('Embed this interactive' not in i.text) and ('xlsx' not in i.text)]

    sects = [sect for sect in soup.find_all('section') if sect.find_all('h2') != []]
    
#     print('sects', sects)
    
    sect = [sect for sect in sects if 'Main points' in sect.find('h2').text]
    
    if len(sect)>0:
                
        sect = sect[0]

        lis = sect.find('ul')
        if lis != None:
            sibling = lis.next_sibling

            while sibling:
                next_sibling = sibling.next_sibling
                sibling.extract()
                sibling = next_sibling
            
        ps = []
        for i in sect:
            if i.name == 'p':
                ps.append(i.text)

            elif i.name == 'ul':
                for j in i.find_all('li'):
                    ps.append(j.text)            
            
    
    else:
        ps = [i.text.replace('\n', '') for i in soup.find_all('p')]
        
    mps = [i for i in ps if ('Source: Office for National Statistics' not in i) and ('Find out more' not in i) and ('make the website work as well as possible' not in i) and ('Embed this interactive' not in i) and ('xlsx' not in i) and ('cookies' not in i)]
    mainpoints = ""
    for i in mps: 
        mainpoints = mainpoints + i + '\n' 
    
    return mainpoints

print(getmps('https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/uklabourmarket/june2021'))